# Zero to Snowflake - Getting Started with Snowflake

**Asset:** Zero to Snowflake - Getting Started with Snowflake  
**Version:** v1  
**Copyright(c):** 2025 Snowflake Inc. All rights reserved.

## 目次
1. 仮想ウェアハウスと設定
2. 永続化クエリ結果の活用
3. 基本的なデータ変換技術
4. UNDROPによるデータ復旧
5. リソースモニター
6. 予算
7. ユニバーサル検索


## セットアップ

開始前に、このクエリを実行してセッションクエリタグを設定します。


In [ ]:
ALTER SESSION SET query_tag = '{"origin":"sf_sit-is","name":"tb_101_v2","version":{"major":1, "minor":1},"attributes":{"is_quickstart":0, "source":"tastybytes", "vignette": "getting_started_with_snowflake"}}';


ワークシートコンテキストを設定することから始めます。データベース、スキーマ、ロールを設定します。


In [ ]:
USE DATABASE tb_101;
USE ROLE accountadmin;


## 1. 仮想ウェアハウスと設定

**ユーザーガイド:**  
https://docs.snowflake.com/en/user-guide/warehouses-overview

仮想ウェアハウスは、Snowflakeデータの分析を実行するために必要な動的で
スケーラブル、かつコスト効率的な計算能力です。その目的は、基盤技術の詳細を
気にする必要なく、すべてのデータ処理ニーズを処理することです。

### ウェアハウスパラメータ:
- **WAREHOUSE_SIZE:** サイズは、ウェアハウス内のクラスターごとに利用可能な計算リソースの量を指定します。利用可能なサイズはX-SmallからEX-Largeまでの範囲です。デフォルト: 'XSmall'
- **WAREHOUSE_TYPE:** 仮想ウェアハウスの種類を定義し、アーキテクチャと動作を決定します。種類: 'STANDARD' 汎用ワークロード用、'SNOWPARK_OPTIMIZED' メモリ集約的なワークロード用。デフォルト: 'STANDARD'
- **AUTO_SUSPEND:** ウェアハウスが自動的にサスペンドする非アクティブ期間を指定します。デフォルト: 600秒
- **INITIALLY_SUSPENDED:** 作成直後にウェアハウスがサスペンド状態で開始するかどうかを決定します。デフォルト: TRUE
- **AUTO_RESUME:** クエリが送信されたときにサスペンド状態から自動的にウェアハウスが再開するかどうかを決定します。デフォルト: TRUE

それでは、最初のウェアハウスを作成しましょう！


まず、アクセス権限のあるアカウント上の既存のウェアハウスを確認しましょう


In [ ]:
SHOW WAREHOUSES;


これにより、ウェアハウスとその属性のリストが返されます：名前、状態（実行中またはサスペンド中）、タイプ、サイズなど多くの情報が含まれます。

Snowsightですべてのウェアハウスを表示・管理することもできます。ウェアハウスページにアクセスするには、ナビゲーションメニューの管理ボタンをクリックし、拡張された管理カテゴリの「ウェアハウス」リンクをクリックします。

ウェアハウスページでは、このアカウント上のウェアハウスとその属性のリストを確認できます。


簡単なSQLコマンドでウェアハウスを作成できます


In [ ]:
CREATE OR REPLACE WAREHOUSE my_wh
    COMMENT = 'My TastyBytes warehouse'
    WAREHOUSE_TYPE = 'standard'
    WAREHOUSE_SIZE = 'xsmall'
    MIN_CLUSTER_COUNT = 1
    MAX_CLUSTER_COUNT = 2
    SCALING_POLICY = 'standard'
    AUTO_SUSPEND = 60
    INITIALLY_SUSPENDED = true,
    AUTO_RESUME = false;


ウェアハウスができたので、このワークシートがこのウェアハウスを使用することを指定する必要があります。これはSQLコマンドまたはUIで実行できます。


In [ ]:
USE WAREHOUSE my_wh;


簡単なクエリを実行してみることができますが、結果ペインにエラーメッセージが表示され、MY_WH ウェアハウスがサスペンド中であることが通知されます。今すぐ試してみてください。


In [ ]:
SELECT * FROM raw_pos.truck_details;


クエリの実行およびすべてのDML操作にはアクティブなウェアハウスが必要ですので、データから洞察を得るためにはウェアハウスを再開する必要があります。

エラーメッセージには、SQLコマンド「ALTER warehouse MY_WH resume」を実行する提案も含まれていました。実行してみましょう！


In [ ]:
ALTER WAREHOUSE my_wh RESUME;


また、ウェアハウスが再度サスペンドした場合に手動で再開する必要がないよう、AUTO_RESUMEをTRUEに設定します。

In [ ]:
ALTER WAREHOUSE my_wh SET AUTO_RESUME = TRUE;

ウェアハウスが実行中になったので、先ほどのクエリを再度実行してみましょう

In [ ]:
SELECT * FROM raw_pos.truck_details;

これでデータに対してクエリを実行できるようになりました

次に、Snowflakeでのウェアハウススケーラビリティの力を見てみましょう。

Snowflakeのウェアハウスはスケーラビリティと弾力性のために設計されており、ワークロードのニーズに基づいて計算リソースを上下に調整する力を提供します。

簡単なALTER WAREHOUSE文でウェアハウスを即座にスケールアップできます。

In [ ]:
ALTER WAREHOUSE my_wh SET warehouse_size = 'XLarge';

それでは、トラックごとの売上を見てみましょう。

In [ ]:
SELECT
    o.truck_brand_name,
    COUNT(DISTINCT o.order_id) AS order_count,
    SUM(o.price) AS total_sales
FROM analytics.orders_v o
GROUP BY o.truck_brand_name
ORDER BY total_sales DESC;

## 2. 永続化クエリ結果の活用

**ユーザーガイド:**  
https://docs.snowflake.com/en/user-guide/querying-persisted-results

続行する前に、Snowflakeの別の強力な機能を実演するのに適した場所です：クエリ結果キャッシュ。

クエリ結果キャッシュの概要:
- すべてのクエリの結果は24時間保持されますが、クエリが実行されるたびにタイマーはリセットされます。
- 結果キャッシュへのヒットはほとんど計算リソースを必要とせず、頻繁に実行されるレポートやダッシュボードに理想的で、クレジット消費の管理に役立ちます。
- キャッシュはクラウドサービス層に存在し、個々のウェアハウスから論理的に分離されています。これにより、同じアカウント内のすべての仮想ウェアハウスとユーザーがグローバルにアクセスできます。

In [ ]:
ALTER WAREHOUSE my_wh SET warehouse_size = 'XSmall';

## 3. 基本的なデータ変換技術

### ゼロコピークローニング

Snowflakeの強力なゼロコピークローニングにより、追加のストレージスペースを使用することなく、データベースオブジェクトの同一で完全に機能する別のコピーを瞬時に作成できます。

In [ ]:
SELECT truck_build FROM raw_pos.truck_details;

In [ ]:
CREATE OR REPLACE TABLE raw_pos.truck_dev CLONE raw_pos.truck_details;

In [ ]:
SELECT TOP 15 * 
FROM raw_pos.truck_dev
ORDER BY truck_id;

新しい列を追加します

In [ ]:
ALTER TABLE raw_pos.truck_dev ADD COLUMN IF NOT EXISTS year NUMBER;
ALTER TABLE raw_pos.truck_dev ADD COLUMN IF NOT EXISTS make VARCHAR(255);
ALTER TABLE raw_pos.truck_dev ADD COLUMN IF NOT EXISTS model VARCHAR(255);

In [ ]:
UPDATE raw_pos.truck_dev
SET 
    year = truck_build:year::NUMBER,
    make = truck_build:make::VARCHAR,
    model = truck_build:model::VARCHAR;

In [ ]:
SELECT year, make, model FROM raw_pos.truck_dev;

In [ ]:
SELECT 
    make,
    COUNT(*) AS count
FROM raw_pos.truck_dev
GROUP BY make
ORDER BY make ASC;

データクリーニング: Ford_をFordに修正

In [ ]:
UPDATE raw_pos.truck_dev
    SET make = 'Ford'
    WHERE make = 'Ford_';

In [ ]:
SELECT truck_id, make 
FROM raw_pos.truck_dev
ORDER BY truck_id;

In [ ]:
ALTER TABLE raw_pos.truck_details SWAP WITH raw_pos.truck_dev;

In [ ]:
SELECT 
    make,
    COUNT(*) AS count
FROM raw_pos.truck_details
GROUP BY make
ORDER BY count DESC;

In [ ]:
ALTER TABLE raw_pos.truck_details DROP COLUMN truck_build;

In [ ]:
DROP TABLE raw_pos.truck_details;

## 4. UNDROPによるデータ復旧

大変です！本番トラックテーブルを誤って削除してしまいました。😱

幸い、UNDROPコマンドを使用してテーブルを削除前の状態に復元できます。

In [ ]:
DESCRIBE TABLE raw_pos.truck_details;

In [ ]:
UNDROP TABLE raw_pos.truck_details;

In [ ]:
SELECT * from raw_pos.truck_details;

In [ ]:
DROP TABLE raw_pos.truck_dev;

## 5. リソースモニター

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/resource-monitors

In [ ]:
USE ROLE accountadmin;

In [ ]:
CREATE OR REPLACE RESOURCE MONITOR my_resource_monitor
    WITH CREDIT_QUOTA = 100
    FREQUENCY = MONTHLY
    START_TIMESTAMP = IMMEDIATELY
    TRIGGERS ON 75 PERCENT DO NOTIFY
             ON 90 PERCENT DO SUSPEND
             ON 100 PERCENT DO SUSPEND_IMMEDIATE;

In [ ]:
ALTER WAREHOUSE my_wh 
    SET RESOURCE_MONITOR = my_resource_monitor;

## 6. 予算

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/budgets

In [ ]:
CREATE OR REPLACE SNOWFLAKE.CORE.BUDGET my_budget()
    COMMENT = 'My Tasty Bytes Budget';

## 7. ユニバーサル検索

**ユーザーガイド:** https://docs.snowflake.com/en/user-guide/ui-snowsight-universal-search

ユニバーサル検索を使用すると、アカウント内のあらゆるオブジェクトを簡単に見つけることができます。

---
## RESET
---

In [ ]:
DROP RESOURCE MONITOR IF EXISTS my_resource_monitor;
DROP TABLE IF EXISTS raw_pos.truck_dev;

In [ ]:
CREATE OR REPLACE TABLE raw_pos.truck_details
AS 
SELECT * EXCLUDE (year, make, model)
FROM raw_pos.truck;

In [ ]:
DROP WAREHOUSE IF EXISTS my_wh;

In [ ]:
ALTER SESSION UNSET query_tag;